In [1]:
!sudo apt install tesseract-ocr -q
!pip install pytesseract -q
!pip install pillow pdf2image -q
!apt-get install -y poppler-utils -q

Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
poppler-utils is already the newest version (22.02.0-2ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [2]:
# mount google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import re
import os
import time

In [8]:
def document_parsing(filepath):
    filename = os.path.basename(filepath).split('.pdf')[0]
    pages = convert_from_path(filepath)
    text_doc = ''
    for page in pages:
        text = pytesseract.image_to_string(page)  # Set language to Indonesian
        text_doc += text

    # uu_perpu -> disahkan
    # pp -> ditetapkan
    # per presiden/keppres -> ditetapkan
    # inpres -> dikeluarkan
    pasal_match = re.search(r'MEMUTUSKAN(.*?)ditetapkan di', text_doc, re.IGNORECASE | re.DOTALL)
    if pasal_match:
        pasal_text = pasal_match.group(1)
    else:
        pasal_text = ""

    topic_text = filename

    doc_dict = {
        "judul": topic_text,
        "isi pasal": pasal_text
    }

    return doc_dict


def extract_text_from_pdfs(input_folder_path, output_folder_path, log_file_path):
    os.makedirs(output_folder_path, exist_ok=True)

    pdf_files = [f for f in os.listdir(input_folder_path) if f.endswith('.pdf')]

    with open(log_file_path, 'w', encoding='utf-8') as log_file:
        for file in pdf_files:
            file_path = os.path.join(input_folder_path, file)
            start_time = time.time()

            doc_dict = document_parsing(file_path)

            output_file = os.path.join(output_folder_path, f'{doc_dict["judul"]}.txt')
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f'Judul: {doc_dict["judul"]}\n\n')
                f.write(f'Isi Pasal:\n{doc_dict["isi pasal"]}')

            end_time = time.time()
            extraction_time = end_time - start_time

            log_file.write(f'Processed file: {file}\n')
            log_file.write(f'Extracted text saved to: {output_file}\n')
            log_file.write(f'Time taken: {extraction_time:.2f} seconds\n')
            log_file.write('-' * 50 + '\n')

            print(f'Processed file: {file}')
            print(f'Extracted text saved to: {output_file}')
            print(f'Time taken: {extraction_time:.2f} seconds')
            print('-' * 50)


In [13]:
!touch "/content/drive/MyDrive/BDC SEMIFINAL/01 Datasets/log_uu_cont.txt"

In [ ]:
# Specify the input folder path containing the PDF files
input_folder_path = '/content/drive/MyDrive/BDC SEMIFINAL/01 Datasets/03 Classified Dataset Manual/UU_Perpu/ditetapkan'
# Specify the output folder path where the extracted text files will be saved
output_folder_path = '/content/drive/MyDrive/BDC SEMIFINAL/01 Datasets/04 txt raw files'
# Specify the log file path
log_file_path = '/content/drive/MyDrive/BDC SEMIFINAL/01 Datasets/log_uu_cont.txt'

extract_text_from_pdfs(input_folder_path, output_folder_path, log_file_path)